In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
"""This file allows you to create a ML model that uses transfer learning and
    vgg16 to classify images """
def main():
  pass

if __name__ == '__main__':

    #Check to see if CUDA is availble on the running systems.
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        print("Using CUDA")

    # Loading this dataset with pytorch is really easy using ImageFolder
    # as the labels are specified by the folders names.
    # This folder path will be custom for each member
    data_dir = r'D:\School\SE\Group_Project\data'
    TRAIN = 'train'
    VAL = 'val'
    TEST = 'test'

    # VGG-16 Takes 224x224 images as input, so we resize all of them
    #it also takes normalization as specified at pytorch.com
    data_transforms = {
        TRAIN: transforms.Compose([
            # Data augmentation is a good practice for the train set
            # Here, we randomly crop the image to 224x224 and
            # randomly flip it horizontally (seems like best fit for graphs?)
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ]),
        VAL: transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ]),
        TEST: transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    }

    #creating our data set
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            transform=data_transforms[x]
        )
        for x in [TRAIN, VAL, TEST]
    }

    #low batch sizes allow for our computers to compute these predictions as they don't use much
    #video memory
    dataloaders = {
        x: torch.utils.data.DataLoader(
            image_datasets[x], batch_size=8,
            shuffle=True, num_workers=4
        )
        for x in [TRAIN, VAL, TEST]
    }

    dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

    for x in [TRAIN, VAL, TEST]:
        print("Loaded {} images under {}".format(dataset_sizes[x], x))


    print("Classes: ")
    class_names = image_datasets[TRAIN].classes
    print(image_datasets[TRAIN].classes)


    #simple function to show images
    def imshow(inp, title=None):
        inp = inp.numpy().transpose((1, 2, 0))
        plt.axis('off')
        plt.imshow(inp)
        if title is not None:
            plt.title(title)
        plt.pause(0.001)
    
    
    #function used in tandem with imshow to
    def show_databatch(inputs, classes):
        out = torchvision.utils.make_grid(inputs)
        imshow(out, title=[class_names[x] for x in classes])
    
    
    inputs, classes = next(iter(dataloaders[TRAIN]))
    show_databatch(inputs, classes)
    
    #allows the integration of matplot lib to see some data
    def visualize_model(vgg, num_images=6):
        was_training = vgg.training
    
        # Set model for evaluation
        vgg.train(False)
        vgg.eval()
    
        images_so_far = 0
    
        for i, data in enumerate(dataloaders[TEST]):
            inputs, labels = data
            size = inputs.size()[0]
    
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
    
            outputs = vgg(inputs)
    
            _, preds = torch.max(outputs.data, 1)
            predicted_labels = [preds[j] for j in range(inputs.size()[0])]
    
            print("Ground truth:")
            show_databatch(inputs.data.cpu(), labels.data.cpu())
            print("Prediction:")
            show_databatch(inputs.data.cpu(), predicted_labels)
    
            del inputs, labels, outputs, preds, predicted_labels
            torch.cuda.empty_cache()
    
            images_so_far += size
            if images_so_far >= num_images:
                break
    
        vgg.train(mode=was_training)  # Revert model back to original training state
    
    #pretraining evaluation
    def eval_model(vgg, criterion):
        since = time.time()
        avg_loss = 0
        avg_acc = 0
        loss_test = 0
        acc_test = 0
    
        test_batches = len(dataloaders[TEST])
        print("Evaluating model")
        print('-' * 10)
    
        for i, data in enumerate(dataloaders[TEST]):
            if i % 100 == 0:
                print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)
    
            vgg.train(False)
            vgg.eval()
            inputs, labels = data
    
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
    
            outputs = vgg(inputs)
    
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
    
            loss_test += loss.data
            acc_test += torch.sum(preds == labels.data)
    
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
    
        avg_loss = loss_test / dataset_sizes[TEST]
        avg_acc = acc_test / dataset_sizes[TEST]
    
        elapsed_time = time.time() - since
        print()
        print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
        print("Avg loss (test): {:.4f}".format(avg_loss))
        print("Avg acc (test): {:.4f}".format(avg_acc))
        print('-' * 10)
    
    
    #Load the pretrained model from pytorch
    vgg16 = models.vgg16_bn()
    
    # Freeze training for all layers
    for param in vgg16.features.parameters():
        param.require_grad = False
    
    # Newly created modules have require_grad=True by default
    num_features = vgg16.classifier[6].in_features
    features = list(vgg16.classifier.children())[:-1]  # Remove last layer
    features.extend([nn.Linear(num_features, len(class_names))])  # Add our layer with 3 outputs
    vgg16.classifier = nn.Sequential(*features)  # Replace the model classifier


    
    # If you want to train the model for more than 2 epochs,
    # set this to True after the first run
    resume_training = False
    
    if resume_training:
        print("Loading pretrained model..")
        vgg16.load_state_dict(torch.load('/Users/matthew/PycharmProjects/CSCI338_Project/VGG16_graphs.pt'))

    
    if use_gpu:
        vgg16.cuda()  # .cuda() will move everything to the GPU side
    
    criterion = nn.CrossEntropyLoss()

    #optimization (possibly could use atom)
    optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    
    
    
    print("Test before training")
    eval_model(vgg16, criterion)
    
    
    visualize_model(vgg16) #test before training

    #(training the actual model, as followed from pytorch)
    def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
        since = time.time()
        best_model_wts = copy.deepcopy(vgg.state_dict())
        best_acc = 0.0

        avg_loss = 0
        avg_acc = 0
        avg_loss_val = 0
        avg_acc_val = 0

        train_batches = len(dataloaders[TRAIN])
        val_batches = len(dataloaders[VAL])

        for epoch in range(num_epochs):
            print("Epoch {}/{}".format(epoch, num_epochs))
            print('-' * 10)

            loss_train = 0
            loss_val = 0
            acc_train = 0
            acc_val = 0

            vgg.train(True)

            for i, data in enumerate(dataloaders[TRAIN]):
                if i % 100 == 0:
                    print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)

                # Use half training dataset
                if i >= train_batches / 2:
                    break

                inputs, labels = data

                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = vgg(inputs)

                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                loss_train += loss.data
                acc_train += torch.sum(preds == labels.data)

                del inputs, labels, outputs, preds
                torch.cuda.empty_cache()

            print()
            # * 2 as we only used half of the dataset
            avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
            avg_acc = acc_train * 2 / dataset_sizes[TRAIN]

            vgg.train(False)
            vgg.eval()

            for i, data in enumerate(dataloaders[VAL]):
                if i % 100 == 0:
                    print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)

                inputs, labels = data

                if use_gpu:
                    inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
                else:
                    inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)

                optimizer.zero_grad()

                outputs = vgg(inputs)

                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                loss_val += loss.data[0]
                acc_val += torch.sum(preds == labels.data)

                del inputs, labels, outputs, preds
                torch.cuda.empty_cache()

            avg_loss_val = loss_val / dataset_sizes[VAL]
            avg_acc_val = acc_val / dataset_sizes[VAL]

            print()
            print("Epoch {} result: ".format(epoch))
            print("Avg loss (train): {:.4f}".format(avg_loss))
            print("Avg acc (train): {:.4f}".format(avg_acc))
            print("Avg loss (val): {:.4f}".format(avg_loss_val))
            print("Avg acc (val): {:.4f}".format(avg_acc_val))
            print('-' * 10)
            print()

            if avg_acc_val > best_acc:
                best_acc = avg_acc_val
                best_model_wts = copy.deepcopy(vgg.state_dict())

        elapsed_time = time.time() - since
        print()
        print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
        print("Best acc: {:.4f}".format(best_acc))

        vgg.load_state_dict(best_model_wts)
        return vgg
    
    #train model
    vgg16 = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=2)

    #take this model and use the data in the remainder of the picture in the testing folder
    torch.save(vgg16.state_dict(), 'VGG16_graphs.pt')


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\School\\SE\\Group_Project\\data/train'